In [6]:
# Batch_first = False인 경우 batch.text.size : (fix_length x 배치크기)
# Batch_first = True인 경우 batch.text.size : (배치크기 x fix_length)

In [1]:
# 데이터 크롤링
import urllib.request
import pandas as pd

urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

df = pd.read_csv('IMDb_Reviews.csv', encoding='latin1') # 데이터를 데이터 프레임에 저장
print(df.head()) # 상위 5개 행 출력
print('전체 샘플의 개수 : {}'.format(len(df)))

# 데이터 추출 + 분리 + 저장
train_df = df[:25000]
test_df = df[25000:]
train_df.to_csv("train_data.csv", index=False) # 훈련 데이터를 csv형태로 저장. index 저장 유무 설정 가능
test_df.to_csv("test_data.csv", index=False) # 테스트 데이터를 csv형태로 저장. index 저장 유무 설정 가능

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x1cca97e8880>)

In [2]:
# 전처리 필드 정의
from torchtext.legacy import data

TEXT = data.Field(sequential = True, use_vocab = True, tokenize = str.split, lower = True, batch_first = True, fix_length = 20)
LABEL = data.Field(sequential = False, use_vocab = False, batch_first = False, is_target = True)

# 데이터셋 제작
from torchtext.legacy.data import TabularDataset

train_data, test_data = TabularDataset.splits(
    path = '.', train='train_data.csv', test='test_data.csv',format='csv',
    fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

In [5]:
# 단어 집합 생성
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)

# 토치텍스트의 데이터로더 생성(배치크기 설정)
from torchtext.legacy.data import Iterator

batch_size = 5 # 배치 사이즈 설정
train_loader = Iterator(dataset=train_data, batch_size=batch_size)
test_loader = Iterator(dataset=test_data, batch_size=batch_size)

batch = next(iter(train_loader)) # 첫번째 미니배치(5개 데이터) 받아옴
print(batch.text)
print(batch.text.shape) # 미니배치의 크기

tensor([[   9,  310,    2,  903,   16,   10,    4,  136,   22,  979,    3, 1950,
         1807, 2340,    9,   14,  979,    3,   25,    5],
        [1422,    2,  131,   14,    3,  107,    0,   18,    9,  126,  387,  112,
            9,  235,    2,  256,  104,    0, 4168,   18],
        [5504,  975, 3041,  137,    5,    3, 1141,  373,  150,  490,  563, 1065,
         1755,    4,   22,  171,    0,  439, 1134,  189],
        [   9,    0,   10,  107,  404,    5,    0,   19,    2,    0,    0,    5,
            0,  320,    0,  171,  106,    2, 7450,    7],
        [ 147,   22,    3,  187,  349,    5,   20,    0,    0,   14,    3, 1088,
          131,    9,  387,   18,    2,   20,   14,    3]])
torch.Size([5, 20])
